In [ ]:
#!/usr/bin/env


"""


split, clean, and self-cal continuum and line data
NOTE: this is intended to be an interactive, iterative process
      so this is more a log that should be run by cutting and
      pasting into casa rather than as an executable script
      search "CHANGEME" for variables to be changed
10/9/15 MCA
"""


#  Setup 

IV_147 M1.0 II
16:00:26.130 -41:53:55.60'


In [ ]:
field = 13                                     # CHANGEME 
file_ms = '../science_calibrated.ms'
contspw   = '2,3,4,7,8,9'                      # continuum spectral windows
contspw_w = [128,3840,1920,128,3840,1920]      # continuum spw widths
robust     = 0.5                               # CHANGEME
imsize     = [640,640]
cell       = '0.03arcsec'
imagermode = 'csclean'
refant     = 'DA52'                            # CHANGEME
xc         = 329                               # CHANGEME
yc         = 315                               # CHANGEME
in_a       = 80
out_a      = 120
aper       = 0.5
boxwidth = 300.
box = rg.box([xc-boxwidth,yc-boxwidth],[xc+boxwidth,yc+boxwidth])


#  Split Off Continuum 

split off field from full ms


In [ ]:
split(vis = file_ms,
      outputvis = 'f'+str(field)+'.vis',
      field = field,
      datacolumn = 'data')


split off continuum (take the large bw spw and average


In [ ]:
split(vis = 'f'+str(field)+'.vis',
      outputvis = 'f'+str(field)+'_cont.vis',
      spw = contspw,
      width = contspw_w,
      datacolumn = 'data')


plot uv-distance vs. amplitude


In [ ]:
plotms(vis='f'+str(field)+'_cont.vis',
       xaxis='uvdist',yaxis='amp',
       coloraxis='spw')
       # plotfile='f'+str(field)+'_ampuv_orig.png'
       # showgui=False,
       # highres=True,
       # overwrite=True)


source is unresolved
find antenna close to center of configuration
check pipeline log that this ant is OK


In [ ]:
plotants(vis='f'+str(field)+'_cont.vis') #, figfile='f'+str(field)+'_ants.png')


#  Clean continuum  

skipping self-cal since source weak
light clean (100 iterations) 
os.system('rm -rf f'+str(field)+'_cont_best*')


In [ ]:
clean(vis = 'f'+str(field)+'_cont.vis',
      imagename = 'f'+str(field)+'_cont_best',
      mode = 'mfs',
      psfmode = 'clark',
      niter = 2000,
      threshold   = '0.0mJy',
      interactive = True,
      mask = '',
      cell        = cell,
      imsize      = imsize,
      weighting   = 'briggs',
      robust      = 0.5,
      imagermode  = imagermode)
im_max = imstat(imagename = 'f'+str(field)+'_cont_best.image')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_cont_best.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
bmaj = imhead(imagename = 'f'+str(field)+'_cont_best.image', mode="get", hdkey="beammajor")
bmin = imhead(imagename = 'f'+str(field)+'_cont_best.image', mode="get", hdkey="beamminor")
print 'Peak = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_max, 1000*im_rms, im_max/im_rms)
print 'Beam = {0:.2f} x {1:.2f} arcsec'.format(bmaj.get('value'),bmin.get('value'))


robust = 0.5
Peak = 1.12 mJy, rms = 0.25 mJy, S/N = 4.5
Beam = 0.34 x 0.28 arcsec
save this to a fits file


In [ ]:
exportfits(imagename='f'+str(field)+'_cont_best.image', fitsimage='f'+str(field)+'_cont.fits')


measure flux
imview(raster=[{'file':'f'+str(field)+'_cont_best.image'}])


In [ ]:
im_rms = imstat(imagename = 'f'+str(field)+'_cont_best.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_cont_best.image',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = 1.13 mJy, rms = 0.25 mJy, S/N = 4.5
re-center image on source and use get_flux.py to get COG flux


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_cont_cropped.image',
             infile  = 'f'+str(field)+'_cont.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_cont_cropped.image',
           fitsimage = 'f'+str(field)+'_cont_cropped.fits')


#  Measure flux with UVMODELFIT 

calculate offset from phase center in arcsec


In [ ]:
pixscale = 0.03             # must match 'cell'                 
dx = pixscale*(320.0-xc)    # offset to east (left)
dy = pixscale*(yc-320.0)    # offset to north (up)


measure flux as gaussian


In [ ]:
uvmodelfit(vis       = 'f'+str(field)+'_cont.vis',
           comptype  = 'G',
           sourcepar = [im_flux,dx,dy,0.5,1,0],
           varypar   = [T,T,T,T,T,T],
           niter     = 10)


In [ ]:
I = 0.00138699 +/- 0.000308009
x = -0.303517 +/- 0.0272066 arcsec
y = -0.174912 +/- 0.0236276 arcsec
a = 0.120558 +/- 0.115307 arcsec
r = 1 +/- 1.51652
p = 0 +/- 57.2958 deg
errors on a too high, using point source method


measure flux as point source


In [ ]:
uvmodelfit(vis       = 'f'+str(field)+'_cont.vis',
           comptype  = 'P',
           sourcepar = [im_flux,dx,dy],
           varypar   = [T,T,T],
           niter     = 10)


In [ ]:
reduced chi2=1.38772
I = 0.00119787 +/- 0.000194544
x = -0.289799 +/- 0.0254975 arcsec
y = -0.16793 +/- 0.0215301 arcsec
consistent with than aperture method
